In [9]:
from PIL import Image, ImageDraw, ImageEnhance, ImageFont, ImageChops, ImageOps
from skimage import io, filters
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import shutil
import cv2
import glob
import random
import ipykernel
import pathlib

Give File Locations


In [ ]:
training_directory = r'C:\Users\Chris\Documents\Uni\Programm for AI\MK3\train'
testing_directory = r'C:\Users\Chris\Documents\Uni\Programm for AI\MK3\test'
valid_directory = r'C:\Users\Chris\Documents\Uni\Programm for AI\MK3\valid'

Create Dataframe

In [6]:

def create_df(directory, dataset_type):
    """
    Create a pandas DataFrame containing information about image files within a specified directory.

    This function scans the specified directory for image files and constructs a DataFrame with the image names,
    their file locations, and the dataset type provided. The function is useful for preparing datasets in machine learning
    projects, especially when handling image data.

    Parameters:
    - directory (str): The path to the directory where image files are stored. The function looks for files with any extension.
    - dataset_type (str): A label indicating the type of dataset. This could be, for example, 'train', 'test', or 'validation'.
    
    Returns:
    - pandas.DataFrame: A DataFrame with three columns: 'image_name', 'file_location', and 'dataset_type'. Each row corresponds
      to an image file found in the specified directory. The 'image_name' column contains the names of the image files,
      'file_location' provides the absolute paths to these files, and 'dataset_type' is filled with the dataset type provided
      as an argument to the function.
      
    Requires:
    - pandas: This function requires the pandas library for creating and manipulating the DataFrame.
    - glob: For fetching the list of files in the directory.
    """
    image_files = glob.glob(os.path.join(directory, '*.*'))
    df = pd.DataFrame({
        'image_name': [os.path.basename(image) for image in image_files],
        'file_location': image_files,
        'dataset_type': dataset_type
    })
    return df

# Create DataFrames for each directory
train_df = create_df(training_directory, 'train')
test_df = create_df(testing_directory, 'test')
valid_df = create_df(valid_directory, 'valid')

# Concatenate all dataframes into one
df = pd.concat([train_df, test_df, valid_df], ignore_index=True)

Add DF collumn for entropy

In [17]:
# Function to calculate entropy
def calculate_entropy(image_path):
    """
    Calculate the entropy of an image based on its grayscale histogram.

    Entropy is a measure of the randomness or unpredictability in the image data, which can be used to quantify the amount
    of information or detail in the image. This function reads an image from a specified path, converts it to grayscale,
    computes its histogram, and then calculates the entropy based on this histogram.

    Parameters:
    - image_path (str): The file path to the image for which entropy is to be calculated. The image is read in grayscale.

    Returns:
    - float: The entropy value of the image. A higher value indicates more complexity or detail in the image.
    
    Requires:
    - skimage.io: For reading the image file.
    - numpy: For numerical calculations, including histogram computation and entropy calculation.
    """
    image = io.imread(image_path, as_gray=True)
    histogram = np.histogram(image, bins=256)[0]
    histogram = histogram / np.sum(histogram)
    histogram = histogram[histogram > 0]
    return -np.sum(histogram * np.log2(histogram))

valid_extensions = '.jpg'
df = df[df['file_location'].apply(lambda x: any(x.endswith(ext) for ext in valid_extensions))]
df['entropy'] = df['file_location'].apply(calculate_entropy)
entropy_bins = [0, 2, 3, 3.5, 4, float('inf')]
df['entropy_category'] = pd.cut(df['entropy'], bins=entropy_bins, labels=False)

Calculate brightness and create new collumn

In [18]:
# Function to calculate brightness
def calculate_brightness(image_path):
    """
    Calculate the average brightness of an image.

    The function converts an image to grayscale and computes its average brightness. Brightness is calculated as the mean
    value of the grayscale intensities.

    Parameters:
    - image_path (str): The file path to the image for which the average brightness is to be calculated.

    Returns:
    - float: The average brightness of the image. Values range from 0 (black) to 255 (white).

    Requires:
    - PIL.Image: For opening and converting the image to grayscale.
    - numpy: For converting the image to an array and calculating the mean brightness.
    """
    image = Image.open(image_path).convert('L')  # convert image to grayscale
    np_image = np.array(image)
    brightness = np.mean(np_image)
    return brightness

# Calculate brightness for each image
df['brightness'] = df['file_location'].apply(calculate_brightness)

# Define brightness categories
brightness_bins = [0, 5, 10, 21, 42, 85, 170, float('inf')]
categories = ['very_dark_brightness', 'dark_brightness', 'extra_low_brightness', 'very_low_brightness', 'low_brightness', 'medium_brightness', 'high_brightness']

# Create brightness category
df['brightness_category'] = pd.cut(df['brightness'], bins=brightness_bins, labels=categories)

C:\Users\Chris\Documents\Uni\Programm for AI\completed test


Augment brightness and contrast based on groups

In [23]:
# Function to adjust brightness and contrast

def adjust_brightness_contrast(input_image_path, brightness=1, contrast=1):
    """
    Adjust the brightness and contrast of an image.

    This function modifies an image's brightness and contrast levels based on the provided parameters. The image is
    overwritten with the adjusted version. Brightness and contrast values are multipliers where 1 means no change,
    values less than 1 decrease the brightness or contrast, and values greater than 1 increase the brightness or contrast.

    Parameters:
    - input_image_path (str): The file path to the image to be adjusted.
    - brightness (float): The multiplier to adjust brightness. Default is 1 (no change).
    - contrast (float): The multiplier to adjust contrast. Default is 1 (no change).

    Requires:
    - PIL.Image: For opening the image file.
    - PIL.ImageEnhance: For adjusting the image brightness and contrast.
    """
    img = Image.open(input_image_path)

    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness)

    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast)

    img.save(input_image_path)
    img.close()

# Initialize an empty list to store the image arrays
image_arrays = []


# Define the brightness and contrast values for each entropy group folder
entropy_group_values = { 
    '0': { 
        "dark_brightness": (1.25, 1.5), 
        "extra_low_brightness": (1, 1.25),
        "high_brightness": (0.75, 1.5), 
        "low_brightness": (1, 1.25), 
        "medium_brightness": (1, 1), 
        "very_dark_brightness": (2.25, 2.5),
        "very_low_brightness": (1, 0.75)  
    },

    '1': { 
        "dark_brightness": (1.75, 1.75),
        "extra_low_brightness": (1.3, 0.75),
        "high_brightness": (0.75, 1.5),
        "low_brightness": (0.75, 1.5),
        "medium_brightness": (1, 1), 
        "very_dark_brightness": (2.25, 2.5), #still to dark, cannot justify increase due to fidelity loss
        "very_low_brightness": (1, 0.75)
    },

    '2': { 
        "dark_brightness": (2, 2),
        "extra_low_brightness": (1.25, 1.25),
        "high_brightness": (0.75, 1.5), 
        "low_brightness": (1, 1.25),
        "medium_brightness": (1, 1), 
        "very_dark_brightness": (2.25, 2.5), 
        "very_low_brightness": (1, 1.25)
    },

    '3': { 
        "dark_brightness": (2, 2),
        "extra_low_brightness": (1.5, 1.25), 
        "high_brightness": (0.75, 1.75), 
        "low_brightness": (1, 1.25), 
        "medium_brightness": (1, 1), 
        "very_dark_brightness": (3, 2),  #still to dark, cannot justify increase due to fidelity loss
        "very_low_brightness": (1, 1.25)
    },

    '4': { 
        "dark_brightness": (2, 2),  
        "extra_low_brightness": (1.5, 1.25), 
        "high_brightness": (0.75, 2), 
        "low_brightness": (1, 1.25), 
        "medium_brightness": (1, 1.15), 
        "very_dark_brightness": (2.25, 2.5), 
        "very_low_brightness": (1, 1.25) 
    }
}

# Iterate over the DataFrame
for index, row in df.iterrows():
    # Get the image path
    image_path = row['file_location']

    # Get the entropy category and brightness group for the image
    entropy_category = str(row['entropy_category'])
    brightness_group = row['brightness_category']

    # Get the brightness and contrast values for the entropy category and brightness group
    brightness, contrast = entropy_group_values[entropy_category][brightness_group]

    # Adjust the brightness and contrast of the image
    adjust_brightness_contrast(image_path, brightness, contrast)


Resize, invert and then overlay the images

In [25]:
# Function to load an image from a file and convert it to an array
def load_image_array(image_path):
    """
    Load an image from a specified path and convert it to a numpy array.

    Parameters:
    - image_path (str): The file path of the image to be loaded.

    Returns:
    - ndarray: A numpy array representing the loaded image.

    Requires:
    - PIL.Image: For opening the image file.
    - numpy: For converting the image to an array.
    """
    img = Image.open(image_path)
    img_array = np.array(img)
    img.close()
    return img_array

# Function to resize an image array
def resize_image_array(img_array):
    """
    Resize an image array to 256x256 pixels.

    Parameters:
    - img_array (ndarray): The numpy array of the image to resize.

    Returns:
    - ndarray: The resized image as a numpy array.

    Requires:
    - PIL.Image: For creating an image object from the numpy array and resizing it.
    - numpy: For converting the resized image back to a numpy array.
    """
    img = Image.fromarray(img_array)
    resized_img = img.resize((256, 256))
    resized_img_array = np.array(resized_img)
    return resized_img_array

# Function to invert an image array
def invert_image_array(img_array):
    """
    Invert the colors of an image array.

    Parameters:
    - img_array (ndarray): The numpy array of the image to invert.

    Returns:
    - ndarray: The color-inverted image as a numpy array.

    Requires:
    - numpy: For performing the inversion operation on the image array.
    """
    inverted_img_array = np.invert(img_array)
    return inverted_img_array

# Function to overlay two image arrays
def overlay_image_arrays(img_array1, img_array2):
    """
    Overlay two image arrays to create a single image showing the lighter parts of both images.

    Parameters:
    - img_array1 (ndarray): The first image array to overlay.
    - img_array2 (ndarray): The second image array to overlay.

    Returns:
    - ndarray: The numpy array of the overlayed image.

    Requires:
    - PIL.Image: For creating image objects from numpy arrays.
    - PIL.ImageChops: For performing the overlay operation.
    - numpy: For converting the overlayed image back to a numpy array.
    """
    img1 = Image.fromarray(img_array1)
    img2 = Image.fromarray(img_array2)
    overlayed_img = ImageChops.lighter(img1, img2)
    overlayed_img_array = np.array(overlayed_img)
    return overlayed_img_array

# Function to save an image array to a file
def save_image_array(img_array, image_path):
    """
    Save an image array to a file.

    Parameters:
    - img_array (ndarray): The image array to save.
    - image_path (str): The file path where the image will be saved.

    Requires:
    - PIL.Image: For creating an image object from the numpy array and saving it.
    """
    img = Image.fromarray(img_array)
    img.save(image_path)

# Iterate over the DataFrame
for index, row in df.iterrows():
    image_array = load_image_array(row['file_location'])
    resized_image_array = resize_image_array(image_array)
    inverted_image_array = invert_image_array(resized_image_array)
    overlayed_image_array = overlay_image_arrays(resized_image_array, inverted_image_array)
    save_image_array(overlayed_image_array, row['file_location'])